# Load Dataset

In [41]:
import pandas as pd


In [55]:
from sklearn.feature_selection import SelectPercentile
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

dataset=pd.read_excel("C:/Users/10253/Desktop/ML-CW-2/Data/trainDataset.xls",index_col=False)
dataset.head(len(dataset)-1)

# c=dataset.corr().abs()
# s=c.unstack()
# so=s.sort_values(kind="quicksort")





# delcol=[]
# c=dataset.corr().shape[0]

# names=dataset.corr().columns
#
# for i in range(c):
#     for j in range(i+1,c):
#         if dataset.corr().iloc[i,j]>0.8 :
#             delcol.append((names[i],names[j]))
# print(len(delcol))
#
# import numpy as np
# np.a
#
# for i in range(c):
#     count = 0
#     for j in range(c):
#         if dataset.corr().iloc[i,j]>0.8:
#             count += 1


y=dataset['pCR (outcome)']
X=dataset.drop('pCR (outcome)',axis=1)

X_train,y_train,X_test,y_test=train_test_split(X,y,test_size=0.3)

select=SelectPercentile(score_func='chi2',percentile=0.9)
X_train=select.fit_transform(X_train,y_train)
X_test=select.transform(X_test)

norm=Normalizer(norm='l1')
X_train=norm.fit_transform(X_train)
X_test=norm.transform(X_test)




pca=PCA(n_components=0.95,svd_solver='full')
newPCA=pca.fit_transform(X)


ValueError: could not convert string to float: 'TRG002629'

In [51]:
dataset['pCR (outcome)'].value_counts()

0      299
1       96
999      5
Name: pCR (outcome), dtype: int64

# Feature Preprocession

# Methods